In [8]:
import pandas as pd
import json
import numpy as np
import fr_core_news_sm
import nltk
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

#nltk.download()
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis.josso\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [162]:
fd = "../0. Data/"


def parse_obj(obj):
    for key in obj:
        if isinstance(obj[key], str):
            obj[key] = obj[key].encode('latin_1').decode('utf-8')
        elif isinstance(obj[key], list):
            obj[key] = list(map(lambda x: x if type(x) != str else x.encode('latin_1').decode('utf-8'), obj[key]))
        pass
    return obj


In [163]:
file = open(fd + 'message_1.json')
data = json.load(file, object_hook=parse_obj)
df = pd.json_normalize(data['messages'])
for i in np.arange(2,6) : 
    file = open(fd + 'message_'+str(i)+'.json', encoding='utf8')
    data = json.load(file)
    df_temp = pd.json_normalize(data['messages'])
    df=df.append(df_temp)
    

df['date_time']=pd.to_datetime(df['timestamp_ms'], unit='ms') 
df['content']=df['content'].str.lower()
df.drop(columns=['timestamp_ms','gifs','is_unsent','photos','type','videos','audio_files','sticker.uri',
                 'call_duration','share.link','share.share_text','users','files'],inplace=True)

df['year']=df['date_time'].dt.year
df=df[df['year']==2021]
df=df[~df['sender_name'].isin(['ThoJean Delavega','Paul Foulonneau','Harry Hrr','Maximilien Waeters'])]

In [164]:
df=df[~df['content'].isna()].reset_index()


In [165]:
df['sender_name'].isin(['ThoJean Delavega','Paul Foulonneau','Harry Hrr','Maximilien Waeters']).sum()

0

In [166]:
final_stopwords_list_1 = stopwords.words('english') + stopwords.words('french')
#final_stopwords_list

In [167]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list_2 = list(fr_stop) + list(en_stop)

In [168]:
nlp = fr_core_news_sm.load()

#temmenized = pd.DataFrame(columns=['sender_name','stem'])
#for i in tqdm(np.arange(df.shape[0]-1)) :
#    doc = nlp(df['content'][i])
#    for token in doc:
#        dict = {'sender_name': df['sender_name'][i], 'stem':token.lemma_}
 #       temmenized=temmenized.append(dict, ignore_index = True)
            

In [170]:
df['parsed_content'] = df['content'].apply(lambda x: [y.lemma_ for y in  nlp(x)])


In [183]:
temmenized=df.explode('parsed_content')[['sender_name','parsed_content']]
temmenized['parsed_content']=temmenized['parsed_content'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [219]:
my_list=['©',',','?','.','!','ã',' ','"','','ã§a','\n','(','etre','avoir','cela','caest','faire']

In [220]:
temmenized_root=temmenized[~(temmenized['parsed_content'].isin(final_stopwords_list_1+my_list))]
temmenized_root['nb_use']=1

c:\users\louis.josso\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [221]:
#temmenized_root[temmenized_root['sender_name']=='Alex Dns'].value_counts().head(10)

In [222]:
temmenized_agg = temmenized_root.groupby(['sender_name','parsed_content']).agg({'nb_use':'count'}).reset_index()
temmenized_agg.sample(3)

,sender_name,parsed_content,nb_use
606,Alex Dns,18h-18h30,1
14365,Thomas Liberge,montre,3
11187,Robin Goutard,gartner,1


In [223]:
temmenized_sorted = temmenized_agg.groupby(['sender_name']).apply(lambda x: x.sort_values(['nb_use'],ascending = False)).reset_index(drop = True)
temmenized_sorted.head(2)

,sender_name,parsed_content,nb_use
0,Adrien Goutard,ouer,18
1,Adrien Goutard,non,16


In [224]:
temmenized_sorted.groupby(['sender_name']).head(2)

,sender_name,parsed_content,nb_use
0,Adrien Goutard,ouer,18
1,Adrien Goutard,non,16
574,Alex Dns,tout,109
575,Alex Dns,aller,99
3138,Alexandre Durand,bon,6
3139,Alexandre Durand,aller,5
3309,Antoine Gilles,;),14
3310,Antoine Gilles,bru,8
3610,Antoine Hamon,;),109
3611,Antoine Hamon,aller,104


In [243]:
word='barbe'
lem_temp=temmenized_sorted[temmenized_sorted['parsed_content']==word]
lem_temp.value_counts()

sender_name             parsed_content  nb_use
Adrien Goutard          barbe           3         1
Alex Dns                barbe           1         1
Antoine Hamon           barbe           2         1
Arthur de Saint-Pierre  barbe           1         1
Louis Jss               barbe           1         1
Robin Goutard           barbe           2         1
dtype: int64

In [248]:
temmenized['sender_name'].value_counts()

Robin Goutard             23688
Louis Jss                 20242
Alex Dns                  14638
Antoine Hamon             11307
Thomas Liberge             9424
Arthur de Saint-Pierre     6074
Adrien Goutard             1902
Antoine Gilles              953
Alexandre Durand            465
Name: sender_name, dtype: int64

In [247]:
temmenized['parsed_content'].value_counts()

le          4078
de          3413
etre        2983
je          2411
avoir       2240
            ... 
penche         1
galice         1
buf            1
amaren         1
toilette       1
Name: parsed_content, Length: 8121, dtype: int64